# House Prediction Regression pipeline with ZenML

ZenML Installation:

In [8]:
%pip install zenml
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for Colab

import IPython

# automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
⠸ Installing integrations...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


{'status': 'ok', 'restart': True}

ZenML Setup:

In [1]:
!rm -rf .zen

In [2]:
!zenml init

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
ZenML repository initialized at /content.
⠹ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.

The local active profile was initialized to 'default' and the local active stack
to 'default'. This local configuration will only take effect when you're running
ZenML from the initialized repository root, or from a subdirectory. For more 
information on profile and stack configuration, please visit 
https://docs.zenml.io/developer-guide/stacks-profiles-repositories.


In [ ]:
!zenml profile create zenbytes

In [3]:
!zenml profile set zenbytes

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Running with active profile: 'default' (local)
Active profile changed to: 'zenbytes'
⠋ Setting the active profile to 'zenbytes'...


In [4]:
!zenml stack set default

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Running with active profile: 'zenbytes' (local)
Active stack set to: 'default'
⠹ Setting the active stack to 'default'...


In [5]:
!zenml stack get

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Running with active profile: 'zenbytes' (local)
The active stack is: 'default'
⠋ Getting the active stack...


Defining house prediction using linear regression pipeline with ZenML:

In [26]:
from zenml.steps import step, Output
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model._base import LinearRegression
%matplotlib inline

In [7]:
@step
def data_loading() -> Output(
    dataset=pd.core.frame.DataFrame,
    ):
    """Load the data"""
    dataset = pd.read_csv("housePrice.csv",header= 0 , sep=',')
    return dataset

In [8]:
@step
def data_preprocessing(dataset: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """Preparing the data"""
    dataset = dataset.dropna(subset =["Address"])
    dataset['Area'] = dataset['Area'].str.replace(',','')
    dataset['Area'] = dataset['Area'].apply(lambda x : float(x[:]))
    dataset = dataset[dataset['Area'] < 1000]
    dataset = dataset[dataset['Parking'].notnull()].copy()
    dataset['Parking'] = dataset['Parking'].astype(int)
    dataset = dataset[dataset['Warehouse'].notnull()].copy()
    dataset['Warehouse'] = dataset['Warehouse'].astype(int)
    dataset = dataset[dataset['Elevator'].notnull()].copy()
    dataset['Elevator'] = dataset['Elevator'].astype(int)
    encoder =LabelEncoder()
    dataset['Address'] = encoder.fit_transform(dataset.Address)
    data_for_set_model =dataset.copy()
    mean =(data_for_set_model.groupby('Address').mean()['Area'])
    mean_price =data_for_set_model.groupby('Address').mean()['Price']
    data_for_set_model['The_value_of_each_address'] =(data_for_set_model['Address'].map(mean_price))/(data_for_set_model['Address'].map(mean))
    data_for_set_model["Room_pre_Area"] =data_for_set_model['Room']/data_for_set_model["Area"]
    return data_for_set_model

In [18]:
@step
def split_the_data(data_for_set_model: pd.core.frame.DataFrame) -> Output(
    train_set=pd.core.frame.DataFrame,
    test_set=pd.core.frame.DataFrame,
    ):
    """Split the data into train and test"""
    data=data_for_set_model.copy()
    train_set , test_set =train_test_split(data , test_size =0.2 , random_state =42)
    return train_set,test_set

In [27]:
@step
def model_training(train_set:pd.core.frame.DataFrame,)-> LinearRegression:
    """Train the Linear regression model"""
    regr = linear_model.LinearRegression()
    x = np.asanyarray(train_set[['The_value_of_each_address','Room','Parking', 'Area']])
    y = np.asanyarray(train_set[['Price']])
    regr.fit (x, y)
    print ('Coefficients: ', regr.coef_)
    print('Intercept: ', regr.intercept_)
    return regr

In [28]:
@step
def model_inference(test_set:pd.core.frame.DataFrame, regr: LinearRegression) -> float:
    """Predict the model"""
    y_hat =regr.predict(test_set[['The_value_of_each_address','Room','Parking','Area']])
    x =np.asanyarray(test_set[['The_value_of_each_address','Room','Parking','Area']])
    y =np.asanyarray(test_set[['Price']])
    print("Residual sum of squares: %.2f" % np.mean((y_hat - y) ** 2))
    print('Variance score: %.2f' % regr.score(x, y))
    return regr.score(x, y)

In [29]:
from zenml.pipelines import pipeline

@pipeline
def house_prediction_pipeline(load, prepare, split, train, predict):
    """Links all the steps together in a pipeline"""
    dataset = load()
    data_for_set_model = prepare(dataset)
    train_set,test_set = split(data_for_set_model)
    model = train(train_set)
    score = predict(test_set, model)

Running zenML Pipeline:

In [30]:
house_prediction_instance = house_prediction_pipeline(
    load=data_loading(), prepare=data_preprocessing(), split=split_the_data(), train=model_training(), predict=model_inference()
)

In [31]:
house_prediction_instance.run()

Creating run for pipeline: house_predictio_pipeline
Cache enabled for pipeline house_predictio_pipeline
Using stack default to run pipeline house_predictio_pipeline...
Step data_loading has started.
Using cached version of data_loading.
Step data_loading has finished in 0.047s.
Step data_preprocessing has started.
Using cached version of data_preprocessing.
Step data_preprocessing has finished in 0.042s.
Step split_the_data has started.
Using cached version of split_the_data.
Step split_the_data has finished in 0.041s.
Step model_training has started.
Coefficients:  [[ 1.29351109e+02  5.27371860e+08 -1.41202432e+09  6.04167129e+07]]
Intercept:  [-6.56554883e+09]
Step model_training has finished in 0.080s.
Step model_inference has started.
Residual sum of squares: 23550024605442904064.00
Variance score: 0.69
Step model_inference has finished in 0.075s.
Pipeline run house_predictio_pipeline-25_Aug_22-12_31_45_178152 has finished in 0.439s.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
